# ETL PARA DATOS DE <font color='teal'>MODELO IA</font>

-----------------------------------------

En el siguiente trabajo, se hará un proceso ETL sobre datos referentes a un modelo de IA que trabaja reconociendo texto, imagen y también transcribiendo audio. Queremos encontrar aquellos datos referentes a detectar las siguientes marcas: Bimbo, Caliente, BARDAHL, Nike, Amazon y Tecate.

In [17]:
# Importamos las librerias necesarias
import json
import pandas as pd

In [18]:
# Usamos google drive para leer sus archivos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
ruta_resultados = '/content/drive/My Drive/etl_datasets/resultados/results.json'

In [20]:
# Leemos el archivo json de la carpeta
with open(ruta_resultados, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Imprime las claves principales
print("Claves principales:", list(data.keys()))

# Dado que la clave principal es 'annotation_results'
# Accedemos a ella para ver su contenido
primer_nivel = data['annotation_results']

# Si es una lista, recórrela
for i, elemento in enumerate(primer_nivel):
    print(f"\nElemento {i}: claves -> {list(elemento.keys())}")
    # Si quieres ver el contenido de una clave específica:
    # print(elemento['nombre_de_la_clave'])

    # Si solo quieres ver un ejemplo y no todo el archivo, puedes hacer un break
    if i == 3:  # Solo muestra los primeros 4 elementos (si los hay)
        break

Claves principales: ['annotation_results']

Elemento 0: claves -> ['input_uri', 'segment', 'segment_label_annotations', 'shot_label_annotations', 'text_annotations', 'logo_recognition_annotations']

Elemento 1: claves -> ['input_uri', 'segment', 'speech_transcriptions']


Observemos que esto ya nos da una pista de como se estructuran los datos en este archivo. Podemos ya incluso ver donde ubicar respuestas del modelo IA respecto a texto, audio e imagen.

In [4]:
with open(ruta_resultados, 'r', encoding='utf-8') as file:
    data = json.load(file)

primer_nivel = data['annotation_results']

for i, elemento in enumerate(primer_nivel):
    print(f"\nElemento {i}: claves -> {list(elemento.keys())}")
    for clave in elemento:
        print(f"  - {clave}: tipo {type(elemento[clave])}")
        # Si quieres ver un ejemplo del contenido:
        if isinstance(elemento[clave], list) and len(elemento[clave]) > 0:
            print(f"    Ejemplo contenido: {elemento[clave][0]}")
        elif isinstance(elemento[clave], dict):
            print(f"    Ejemplo contenido: {list(elemento[clave].items())[:1]}")
        else:
            print(f"    Valor: {elemento[clave]}")
    if i == 2:  # Solo muestra los primeros 3 elementos
        break


Elemento 0: claves -> ['input_uri', 'segment', 'segment_label_annotations', 'shot_label_annotations', 'text_annotations', 'logo_recognition_annotations']
  - input_uri: tipo <class 'str'>
    Valor: /vboxio-off/partidos/2025/TUDN/apertura/jornada_02/AMERICA_TIJUANA/16-07-2025_TU_APE_J02_AMETIJ.mkv
  - segment: tipo <class 'dict'>
    Ejemplo contenido: [('start_time_offset', {})]
  - segment_label_annotations: tipo <class 'list'>
    Ejemplo contenido: {'entity': {'entity_id': '/m/02vx4', 'description': 'soccer', 'language_code': 'en-US'}, 'category_entities': [{'entity_id': '/m/06ntj', 'description': 'sports', 'language_code': 'en-US'}], 'segments': [{'segment': {'start_time_offset': {}, 'end_time_offset': {'seconds': 8133, 'nanos': 933333000}}, 'confidence': 0.70713681}]}
  - shot_label_annotations: tipo <class 'list'>
    Ejemplo contenido: {'entity': {'entity_id': '/m/02qdwbp', 'description': 'net', 'language_code': 'en-US'}, 'segments': [{'segment': {'start_time_offset': {'second

Procedemos a restringir la información según las marcas que nos interesa observar.

In [21]:
# Definimos las marcas de interes
marcas_interes = ['bimbo', 'caliente', 'bardahl', 'nike', 'amazon', 'tecate']

# Forzamos a la lectura de archivos a encontrar coincidencias de
# palabras de marcas de interes a traves de las llaves
with open(ruta_resultados, 'r', encoding='utf-8') as file:
    data = json.load(file)

resultados = []
for elemento in data['annotation_results']:
    input_uri = elemento.get('input_uri', '')
    if 'text_annotations' in elemento:
        for texto in elemento['text_annotations']:
            texto_detectado = texto.get('text', '')
            # Normaliza el texto: minúsculas y separa por palabras
            palabras = [palabra.strip().lower() for palabra in texto_detectado.split()]
            # Busca si alguna marca está en las palabras detectadas
            marcas_encontradas = [marca for marca in marcas_interes if marca in palabras]
            for marca in marcas_encontradas:
                resultados.append({
                    'input_uri': input_uri,
                    'marca_detectada': marca,
                    'texto_original': texto_detectado
                })

# Crear DataFrame
df = pd.DataFrame(resultados)
print(df.head())

                                           input_uri marca_detectada  \
0  /vboxio-off/partidos/2025/TUDN/apertura/jornad...          amazon   
1  /vboxio-off/partidos/2025/TUDN/apertura/jornad...           bimbo   
2  /vboxio-off/partidos/2025/TUDN/apertura/jornad...        caliente   
3  /vboxio-off/partidos/2025/TUDN/apertura/jornad...         bardahl   
4  /vboxio-off/partidos/2025/TUDN/apertura/jornad...         bardahl   

                                      texto_original  
0  15 al 21 de julio Grandes ofertas amazon prime...  
1                                         cond BIMBO  
2               BARDAHL C caliente.mx 6 caliente TIX  
3               BARDAHL C caliente.mx 6 caliente TIX  
4                       BARDAHL e caliente.mx reSTOI  


Ahora, necesitamos crear funciones que nos ayuden a medir como fueron las detecciones del modelo de IA segun sea el formato (audio, texto, imagen). Requerimos de usar otras llaves complemetarias en el archivo JSON que nos permitiran calcular las variables de Tiempo de detección, Duración y Área. Todo esto se agregrará a nuestra tabla estructurada final.

In [22]:
# Funciones para ordenar valores en tabla estructurada
def calcular_area(vertices):
    # Solo toma vértices que tengan 'x' y 'y'
    xs = [v['x'] for v in vertices if 'x' in v and 'y' in v]
    ys = [v['y'] for v in vertices if 'x' in v and 'y' in v]
    if not xs or not ys:
        return None
    ancho = max(xs) - min(xs)
    alto = max(ys) - min(ys)
    return round(ancho * alto * 100, 2)  # porcentaje

def calcular_area_logo(bbox):
    # Verifica que existan todas las claves necesarias
    if not all(k in bbox for k in ['left', 'top', 'right', 'bottom']):
        return None
    ancho = bbox['right'] - bbox['left']
    alto = bbox['bottom'] - bbox['top']
    return round(ancho * alto * 100, 2)  # porcentaje

def offset_a_segundos(offset):
    if not offset:
        return 0
    return offset.get('seconds', 0) + offset.get('nanos', 0)/1e9

with open(ruta_resultados, 'r', encoding='utf-8') as file:
    data = json.load(file)

resultados = []

for elemento in data['annotation_results']:
    input_uri = elemento.get('input_uri', '')

    # --- TEXTO (OCR) ---
    for texto in elemento.get('text_annotations', []):
        texto_detectado = texto.get('text', '')
        palabras = [palabra.strip().lower() for palabra in texto_detectado.split()]
        marcas_encontradas = [marca for marca in marcas_interes if marca in palabras]
        for marca in marcas_encontradas:
            for segmento in texto.get('segments', []):
                seg = segmento.get('segment', {})
                inicio = offset_a_segundos(seg.get('start_time_offset', {}))
                fin = offset_a_segundos(seg.get('end_time_offset', {}))
                duracion = max(0, fin - inicio)
                area = None
                # Buscar área en frames (si existe)
                frames = segmento.get('frames', [])
                if frames:
                    # Tomar el área del primer frame como ejemplo
                    vertices = frames[0].get('rotated_bounding_box', {}).get('vertices', [])
                    if vertices:
                        area = calcular_area(vertices)
                resultados.append({
                    'Marca': marca,
                    'Tiempo de detección': inicio,
                    'Duración': duracion,
                    'Área': area,
                    'Modo de detección': 'texto',
                    'input_uri': input_uri
                })

    # --- LOGOS ---
    for logo in elemento.get('logo_recognition_annotations', []):
        marca_logo = logo.get('entity', {}).get('description', '').strip().lower()
        if marca_logo in marcas_interes:
            for track in logo.get('tracks', []):
                seg = track.get('segment', {})
                inicio = offset_a_segundos(seg.get('start_time_offset', {}))
                fin = offset_a_segundos(seg.get('end_time_offset', {}))
                duracion = max(0, fin - inicio)
                area = None
                # Buscar área en timestamped_objects
                objs = track.get('timestamped_objects', [])
                if objs:
                    bbox = objs[0].get('normalized_bounding_box', {})
                    if bbox:
                        area = calcular_area_logo(bbox)
                resultados.append({
                    'Marca': marca_logo,
                    'Tiempo de detección': inicio,
                    'Duración': duracion,
                    'Área': area,
                    'Modo de detección': 'logo',
                    'input_uri': input_uri
                })

    # --- AUDIO ---
    for transcripcion in elemento.get('speech_transcriptions', []):
        for alternativa in transcripcion.get('alternatives', []):
            transcript = alternativa.get('transcript', '').lower()
            for marca in marcas_interes:
                if marca in transcript:
                    # Buscar la palabra y su tiempo
                    for palabra in alternativa.get('words', []):
                        if marca in palabra.get('word', '').lower():
                            inicio = offset_a_segundos(palabra.get('start_time', {}))
                            fin = offset_a_segundos(palabra.get('end_time', {}))
                            duracion = max(0, fin - inicio)
                            resultados.append({
                                'Marca': marca,
                                'Tiempo de detección': inicio,
                                'Duración': duracion,
                                'Área': None,
                                'Modo de detección': 'audio',
                                'input_uri': input_uri
                            })

# Crear DataFrame final
df = pd.DataFrame(resultados)
print(df.head())

      Marca  Tiempo de detección  Duración  Área Modo de detección  \
0    amazon               2981.4       0.0  2.62             texto   
1     bimbo               1678.2       0.4  0.80             texto   
2  caliente               4780.8       0.0  3.33             texto   
3   bardahl               4780.8       0.0  3.33             texto   
4   bardahl               5687.6       0.0  3.72             texto   

                                           input_uri  
0  /vboxio-off/partidos/2025/TUDN/apertura/jornad...  
1  /vboxio-off/partidos/2025/TUDN/apertura/jornad...  
2  /vboxio-off/partidos/2025/TUDN/apertura/jornad...  
3  /vboxio-off/partidos/2025/TUDN/apertura/jornad...  
4  /vboxio-off/partidos/2025/TUDN/apertura/jornad...  


In [23]:
df.head()

,Marca,Tiempo de detección,Duración,Área,Modo de detección,input_uri
0,amazon,2981.4,0.0,2.62,texto,/vboxio-off/partidos/2025/TUDN/apertura/jornad...
1,bimbo,1678.2,0.4,0.80,texto,/vboxio-off/partidos/2025/TUDN/apertura/jornad...
2,caliente,4780.8,0.0,3.33,texto,/vboxio-off/partidos/2025/TUDN/apertura/jornad...
3,bardahl,4780.8,0.0,3.33,texto,/vboxio-off/partidos/2025/TUDN/apertura/jornad...
4,bardahl,5687.6,0.0,3.72,texto,/vboxio-off/partidos/2025/TUDN/apertura/jornad...


Procedemos a eliminar la columna que no necesitamos

In [25]:
df = df.drop(['input_uri'], axis=1)

In [26]:
df.head()

,Marca,Tiempo de detección,Duración,Área,Modo de detección
0,amazon,2981.4,0.0,2.62,texto
1,bimbo,1678.2,0.4,0.80,texto
2,caliente,4780.8,0.0,3.33,texto
3,bardahl,4780.8,0.0,3.33,texto
4,bardahl,5687.6,0.0,3.72,texto


In [27]:
df.shape

(9211, 5)

Luego, revisamos valores nulos.

In [11]:
df.isnull().sum().sort_values(ascending=False) / df.shape[0]

,0
Área,0.000543
Marca,0.000000
Tiempo de detección,0.000000
Duración,0.000000
Modo de detección,0.000000


Imputamos los valores nulos con la media

In [28]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
df['Área'] = imputer.fit_transform(df[['Área']])

In [29]:
df.isnull().sum().sort_values(ascending=False) / df.shape[0]

,0
Marca,0.0
Tiempo de detección,0.0
Duración,0.0
Área,0.0
Modo de detección,0.0


Finalmente, guardamos la nueva tabla estructurada con los datos ya transformados.

In [30]:
df.to_csv('tabla_ia_estructurada.csv')